# Diffusion Probabilistic Models

Revisión informal de "_Deep Unsupervised Learning using Nonequilibrium Thermodynamics_".

Técnica publicada en marzo de 2015.

- [https://arxiv.org/pdf/1503.03585](https://arxiv.org/pdf/1503.03585)

Trabajo fundacional que estableció las bases teóricas de los modelos generativos de difusión.

## 1. Arquitectura

Un modelo generativo genera valores que siguen una determinada distribución de probabilidad de referencia dada. El reto es que el modelo sea capaz de generar cualquier distribución dada, por muy compleja que sea.

Los modelos generativos de difusión utilizan una técnica basada en convertir la distribución de referencia compleja en otra más sencilla. Y lo hacen aplicando ruido gradualmente a los datos originales hasta que tengan una distribución sencilla.

```
Data ─> Diffusion ─> Noise
```

Y enseñando al modelo a revertir el proceso, generando datos estructurados con la distribución compleja a partir de ruido con una distribución sencilla.

```
Noise ─> Inverse Diffusion ─> Data
```

El _paper_ detalla un proceso de difusión, uno de difusión inversa, y cómo utilizar ambos procesos para entrenar un modelo.

## 2. Diffusion

El proceso de difusión destruye gradualmente la estructura de los datos originales añadiendo ruido de manera progresiva en $T$ pasos.

```
Data ─> T x Diffusion ─> Noise
```

El proceso se presenta de manera formal, y posteriormente se aclara cómo y cuándo aplicarlo.

### 2.1. Forward Trajectory

El marco teórico del _paper_ parte de una distribución de probabilidad compleja.

- $ q(\mathbf{x}^{(0)}) $

Con el objetivo de convertirla en otra distribución más sencilla.

- $ \pi(\mathbf{y}) $

Mediante aplicaciones sucesivas de un núcleo de difusión de Markov.

- $ \mathcal{T}_{\pi}(\mathbf{y} \mid \mathbf{y'} ; \beta) $

La idea es que se puede avanzar desde la distribución original hasta la deseada (_Forward Trajectory_) en un número finito de pasos.

- $ q \left( \mathbf{x}^{(0 \ldots T)} \right) = q \left( \mathbf{x}^{(0)} \right) \prod\limits_{t=1}^T q \left( \mathbf{x}^{(t)} \mid \mathbf{x}^{(t-1)} \right) $

Estando los pasos para transicionar de una distribución a otra definidos por el núcleo de difusión de Markov.

- $ q \left( \mathbf{x}^{(t)} \mid \mathbf{x}^{(t-1)} \right) = \mathcal{T}_{\pi} \left(\mathbf{x}^{(t)} \mid \mathbf{x}^{(t-1)} ; \beta_t \right) $

Y siendo la distribución deseada una distribución _estacionaria_, que una vez alcanzada no se abandona (condición de equilibrio).

- $ \pi(\mathbf{y}) = \displaystyle \int \pi(\mathbf{y'}) \mathcal{T}_{\pi} (\mathbf{y} \mid \mathbf{y'} ; \beta) d\mathbf{y'} $

Siendo:

- $\mathbf{x}^{(0)}$: los datos de partida, o estado inicial, por ejemplo una imagen.

- $q(\mathbf{x}^{(0)})$: la distribución de probabilidad de los datos de partida, habitualmente compleja y analíticamente intratable.

- $q \left( \mathbf{x}^{(t)} \mid \mathbf{x}^{(t-1)} \right)$: la probabilidad de transicionar al estado $\mathbf{x}^{(t)}$ desde el estado anterior $\mathbf{x}^{(t-1)}$.

- $\pi(\mathbf{y})$: la distribución de probabilidad deseada, más sencilla y tratable que la de partida, como por ejemplo una gaussiana con matriz de covarianza igual a la identidad (gaussiana isótropa).

- $\mathcal{T}_{\pi}(\mathbf{y} \mid \mathbf{y'} ; \beta)$: el núcleo de difusión que define la probabilidad de pasar de un estado a otro en un solo paso.

- $\beta_t$: la tasa de difusión, o cantidad de ruido, aplicada en el paso $t$.

- $q \left( \mathbf{x}^{(0 \ldots T)} \right)$: la probabilidad conjunta de la trayectoria completa, desde el estado inicial hasta el final, en $T$ pasos.

### 2.2. Distribución Estacionaria

Antes de continuar, es interesante detenerse en la expresión de la distribución estacionaria.

- $ \pi(\mathbf{y}) = \displaystyle \int \pi(\mathbf{y'}) \mathcal{T}_{\pi} (\mathbf{y} \mid \mathbf{y'} ; \beta) d\mathbf{y'} $

Siendo:

- $\pi(\mathbf{y})$: la probabilidad de que el sistema se encuentre en el estado $\mathbf{y}$.

- $\pi(\mathbf{y'})$: la probabilidad de que el sistema se encuentre en cualquier otro estado $\mathbf{y'}$.

- $\mathcal{T}_{\pi} (\mathbf{y} \mid \mathbf{y'} ; \beta)$: la probabilidad de transicionar del estado $\mathbf{y'}$ al estado $\mathbf{y}$.

- $\pi(\mathbf{y'}) \mathcal{T}_{\pi} (\mathbf{y} \mid \mathbf{y'} ; \beta)$: la probabilidad conjunta de que el sistema se encuentre en el estado $\mathbf{y'}$ y transicione al estado $\mathbf{y}$.

- $\displaystyle \int \ldots d\mathbf{y'}$: la integral suma estas probabilidades conjuntas sobre todos los posibles estados de partida $\mathbf{y'}$.

Es decir, la probabilidad de que el sistema se encuentre en el estado $\mathbf{y}$ es igual a la probabilidad total de que transicione a dicho estado desde cualquier otro estado posible.

La distribución de probabilidad no cambia después de transicionar. El sistema ha alcanzado un estado de equilibrio y la distribución $\pi$ se mantiene estable.

### 2.3. Diffusion Kernel

En el proceso de difusión (_Forward Trajectory_) se utiliza un núcleo de difusión concreto con una forma predefinida.

En un apéndice del _paper_ se muestran las ecuaciones utilizadas para los experimentos asumiendo que la distribución estacionaria es gaussiana, con media cero y covarianza igual a la identidad.

- $ \pi \left( \mathbf{x}^{(T)} \right) = \mathcal{N} \left( \mathbf{x}^{(T)}; \mathbf{0}, \mathbf{I} \right) $

De forma que el núcleo de difusión es también una distribución gaussiana.

- $ q \left( \mathbf{x}^{(t)} \mid \mathbf{x}^{(t-1)} \right) = \mathcal{N} \left( \mathbf{x}^{(t)}; \mathbf{x}^{(t-1)} \sqrt{1 - \beta_t}, \mathbf{I} \beta_t \right) $

El núcleo añade ruido progresivamente, aumentando la covarianza por $\beta_t$. Y disminuye la influencia de la señal original progresivamente, atenuando la media por $\sqrt{1 - \beta_t}$.

Si $\beta_t$ es cercano a cero la señal original se mantiene en gran medida. Si es cercano a uno se atenúa casi completamente y lo que queda principalmente es ruido gaussiano.

En el apéndice del _paper_ se incluyen también las ecuaciones para una distribución binomial.

### 2.4. Diffusion Rates

En cada paso del proceso de difusión se aplica una tasa de difusión $\beta_t$ distinta, indicando la cantidad de ruido a añadir en cada paso $t$.

En el _paper_ se sugiere que estos valores pueden ser aprendidos, pero también pueden ser fijos.

Propone un esquema sencillo para las distribuciones binomiales, que también debería funcionar para las gaussianas.

- $ \beta_t = \dfrac{1}{T - t + 1} $

Lo que genera una secuencia de valores fijos desde $1$ hasta $T$.

- $ \dfrac{1}{T}$, $\dfrac{1}{T -1}$, $\dfrac{1}{T -2}$, $\ldots$, $1 $

Empieza con un valor pequeño que se va incrementando en cada paso $t$ hastar llegar a $1$ en el último paso $T$.

Lo que significa que se añade poco ruido al principio y mucho más en los pasos finales del proceso de difusión.

## 3. Inverse Diffusion

El proceso de difusión inversa genera valores que siguen la estructura de los datos originales a partir de ruido en $T$ pasos.

```
Noise ─> T x Inverse Diffusion ─> Data
```

El proceso se presenta de manera formal, y posteriormente se aclara cómo entrenar un modelo para que lo aprenda.

### 3.1. Reverse Trajectory

La idea es calcular una nueva distribución de probabilidad que sea igual que la estacionaria para el último estado de la cadena.

- $ p \left( \mathbf{x}^{(T)}\right ) = \pi \left( \mathbf{x}^{(T)} \right) $

Y que permita retroceder desde la distribución estacionaria hasta la original (_Reverse Trajectory_) en el mismo número de pasos.

- $ p \left( \mathbf{x}^{(0 \ldots T)} \right) = p \left( \mathbf{x}^{(T)} \right) \prod\limits_{t=1}^T p \left( \mathbf{x}^{(t-1)} \mid \mathbf{x}^{(t)} \right) $

Siendo la probabilidad de transición inversa la que el modelo debe aprender.

- $ p \left( \mathbf{x}^{(t-1)} \mid \mathbf{x}^{(t)} \right) $

El problema es que esta distribución es computacionacionalmente intratable. Requiere integrar sobre todas las distribuciones de datos originales. Por lo que el _paper_ propone entrenar un modelo que aprenda a aproximarla.

Y además demuestra que la transición inversa tiene la misma forma gaussiana que la transición directa, si se toman pasos de difusión $\beta_t$ suficientemente pequeños. Lo que permite diseñar un modelo que aprenda una distribución gaussiana.

### 3.2. Diffusion Kernel

El núcleo de difusión inversa es el que el modelo debe aprender para revertir el proceso de difusión paso a paso.

Se parametriza como una distribución gaussiana, donde la media y la covarianza son aprendidas por el modelo.

- $ p \left( \mathbf{x}^{(t-1)} \mid \mathbf{x}^{(t)} \right) = \mathcal{N} \left( \mathbf{x}^{(t-1)}; \mathbf{f}_{\mu} (\mathbf{x}^{(t)}, t), \mathbf{f}_{\Sigma} (\mathbf{x}^{(t)}, t) \right) $

Las funciones $\mathbf{f}_{\mu}$ y $\mathbf{f}_{\Sigma}$ representan el modelo, que aprende a predecir los parámetros de la gaussiana a partir del dato con ruido $\mathbf{x}^{(t)}$ y el paso de tiempo $t$.

En un apéndice del _paper_ se incluyen también las ecuaciones para una distribución binomial.

## 4. Entrenamiento

Una vez presentados formalmente los procesos de difusión y difusión inversa, queda la tarea de diseñar la forma de entrenar un modelo utilizando ambos procesos.

Se plantea qué muestras utilizar para el entrenamiento, con qué compararlas, y cómo definir una función de pérdida óptima que mida la bondad del modelo.

### 4.1. Esquema General

```
Input ─> Model ─> Output
```

Entradas:

- $\mathbf{x}^{(t)}$: una muestra de datos con ruido, correspondiente al paso de tiempo $t$, con dimensiones acorde a la naturaleza de los datos sobre los que se realice el entrenamiento.

- $t$: el paso de tiempo actual, un valor escalar entre $1$ y $T$.

Salidas:

- $\mathbf{f}_{\mu} (\mathbf{x}^{(t)}, t)$: media de la distribución de probabilidad estimada por el modelo, con las mismas dimensiones que la entrada $\mathbf{x}^{(t)}$.

- $\mathbf{f}_{\Sigma} (\mathbf{x}^{(t)}, t)$: covarianza de la distribución de probabilidad estimada por el modelo, representada por un vector, para covarianza diagonal, o un escalar, para varianza isótropa.

La arquitectura específica de la red es indiferente, pudiéndose utilizar aquella que mejor se adapte a la naturaleza de los datos sobre los que se realiza el entrenamiento. Desarrollos posteriores han mejorado la forma original propuesta de diseñar y entrenar la red.

### 4.2. Noised Samples

En este punto se plantea "_¿qué muestras utilizar para el entrenamiento?_".

```
Input ─> Difussion ─> Model ─> Output
```

El modelo se entrena con muestras del _dataset_ de entrenamiento a las que se le aplica el proceso de difusión.

- Se toma una muestra $\mathbf{x}^{(0)}$ del _dataset_.

- Se genera un número aleatorio $t$ entre $1$ y $T$ que representa un paso de tiempo.

- Y se genera la muestra $\mathbf{x}^{(t)}$ aplicando el proceso de difusión a $\mathbf{x}^{(0)}$ hasta el paso $t$.

Dependiendo de la arquitectura de la red, las tasas de difusión $\beta$ pueden ser fijas, o aprendidas por el modelo.

```python
t = rand(1, T)

x_t = noised_sample(x_0, t, betas)
```

### 4.3. Reparameterization Trick

En este punto se plantea "_¿hay una forma eficiente de aplicar ruido a las muestras de entrada?_".

```
Input ─> Reparameterization Trick ─> Model ─> Output
```

Para calcular $\mathbf{x}^{(t)}$ desde $\mathbf{x}^{(0)}$ hay que aplicar el núcleo de difusión $t$ veces de forma iterativa, lo que puede ser computacionalmente ineficiente.

Afortunadamente, una propiedad de las distribuciones gaussianas es que la suma de varias de ellas es también una gaussiana.

Una muestra con ruido $\mathbf{x}^{(t)}$ se puede expresar en función de la muestra original $\mathbf{x}^{(0)}$ y un **único** ruido gaussiano $\boldsymbol{\epsilon}$.

- $ \mathbf{x}^{(t)} = \sqrt{\bar\alpha_t} \mathbf{x}^{(0)} + \sqrt{1 - \bar\alpha_t} \boldsymbol{\epsilon} $

Siendo $\bar{\alpha}_t$ el producto acumulado de $(1 - \beta_i)$ hasta el tiempo $t$.

- $ \alpha_t = 1 - \beta_t $

- $ \bar\alpha_t = \prod\limits_{i=1}^{t} \alpha_i $

Si $\beta_t$ es la cantidad de ruido que se añade en cada paso, $\alpha_t$ representa la cantidad de la señal que se conserva.

Lo que quiere decir todo esto, es que durante el entrenamiento no se toman las muestras del _dataset_ y se itera sobre ellas aplicando el núcleo de difusión un número $t$ de pasos deseados.

```python
x_t = x_0

for i in [1..t]:
  mean = sqrt(1 - betas[i])
  std = sqrt(betas[i])
  eps = randn()

  x_t = x_t * mean + std * eps
```

Lo que se hace es aplicar directamente el _reparameterization trick_ y de una sola vez calcular el resultado para el paso $t$ deseado.

```python
alpha_bar_t = cumprod(1 - betas)[t]
eps = randn()

x_t = sqrt(alpha_bar_t) * x_0 + sqrt(1 - alpha_bar_t) * eps
```

Los valores $\beta_t$ y $\bar\alpha_t$ se pueden precalcular, lo que hace el entrenamiento computacionalmente factible.

### 4.4. Ground Truth

En este punto se plantea "_¿qué valores utilizar para comprobar cuan buenas son las predicciones del modelo?_".

```
Input ─> Reparameterization Trick ─> Model ─> Output ─> vs <─ Ground Truth
```

Lo que el modelo debe aprender es el proceso de difusión inversa, es decir, la probabilidad de transicionar al estado $\mathbf{x}^{(t-1)}$ desde el estado $\mathbf{x}^{(t)}$.

- $ p \left( \mathbf{x}^{(t-1)} \mid \mathbf{x}^{(t)} \right) $

Y lo que se plantea es comprobar la bondad de las predicciones del modelo utilizando una distribución _posterior_ como _verdad fundamental_ (_Ground Truth_).

La probabilidad "_verdadera_" de llegar al estado $\mathbf{x}^{(t-1)}$ desde el estado $\mathbf{x}^{(t)}$ conociendo el estado inicial $\mathbf{x}^{(0)}$.

- $ q \left( \mathbf{x}^{(t-1)} \mid \mathbf{x}^{(t)}, \mathbf{x}^{(0)} \right) $

El modelo no conoce el estado inicial, es lo que aprende a predecir, por lo que esta distribución posterior es óptima para medir la bondad del modelo. Guía al modelo para que sea capaz de predecir cómo es la distribución original de los datos.

Distribución que según el _paper_ resulta ser una gaussiana con media:

- $ \bar{\mu}_t(\mathbf{x}^{(t)}, \mathbf{x}^{(0)}) = \dfrac{\sqrt{\bar\alpha_{t-1}} \beta_t}{1 - \bar{\alpha}_t} \mathbf{x}^{(0)} + \dfrac{\sqrt{\alpha_t} (1 - \bar\alpha_{t-1})}{1 - \bar\alpha_t} \mathbf{x}^{(t)} $

Y varianza:

- $ \bar{\beta}_t = \dfrac{1 - \bar\alpha_{t-1}}{1 - \bar\alpha_t} \beta_t $

Es decir, que se pueden tomar los valores del _dataset_, aplicarles el proceso de difusión de forma sencilla utilizando el _reparameterization trick_, y calcular a partir de ellos también de forma sencilla la media y varianza verdaderas para dichos valores.

```python
true_mean = (sqrt(alpha_bar_t_minus_1) * beta_t * x_0 +  sqrt(alpha_t) * (1 - alpha_bar_t_minus_1) * x_t) / (1 - alpha_bar_t)

true_variance = ((1 - alpha_bar_t_minus_1) * beta_t) / (1 - alpha_bar_t)
```

### 4.5. Loss Function

En este punto se plantea "_¿qué función de pérdida utilizar para comparar los valores esperados con los obtenidos?_".

```
Input ─> Reparameterization Trick ─> Model ─> Output ─> Loss Function <─ Ground Truth
```

El objetivo es que la distribución predicha por el modelo:

- $ p \left( \mathbf{x}^{(t-1)} \mid \mathbf{x}^{(t)} \right) $

Sea lo más parecida posible a la distribución _verdadera_:

- $ q(\mathbf{x}^{(t-1)} \mid \mathbf{x}^{(t)}, \mathbf{x}^{(0)}) $

Para medir la diferencia entre las dos probabilidades se utiliza la _Divergencia KL_ (_Kullback–Leibler Divergence_) como función de pérdida.

- $ D_{KL}(P \Vert Q) = \displaystyle \sum\limits_{x \in \mathcal{X}} P(x) log \dfrac{P(x)}{Q(x)} $

Siendo $P$ la distribución verdadera y $Q$ la distribución aprendida (exactamente la nomenclatura contraria que utiliza el _paper_).

- $ L_t = D_{KL}(q(\mathbf{x}^{(t-1)} \mid \mathbf{x}^{(t)}, \mathbf{x}^{(0)}) \Vert p(\mathbf{x}^{(t-1)} \mid \mathbf{x}^{(t)})) $

Y lo importante es que se demuestra, que para distribuciones gaussianas, la _Divergencia KL_ tiene una solución analítica y puede por tanto calcularse.

```python
f_mu, f_sigma = model(x_t, t)

q = Normal(true_mean, sqrt(true_variance))
p = Normal(f_mu, sqrt(f_sigma))

loss = mean(sum(kl_divergence(q, p)))
```